# Evidently UI Tutorial

## Working with Local Workspace

First, let’s create a local workspace where all our Evidently projects and reports will be stored. This is essentially a directory to keep things organized.


In [ ]:
from evidently.ui.workspace import Workspace

ws = Workspace.create("workspace")

### Create a New Project

Now, inside our workspace, we’ll create a new project. Projects in Evidently are containers for organizing related reports and monitoring runs.


In [ ]:
project = ws.create_project("My Project")
project

this creates project dir in `workspace/{project.id}`

### Working with projects

`Workspace` has some methods to work with projects.

For example, we can edit project to add a description to clarify projects purpose or context.


In [ ]:
project.description = "Evidently Service example project"
ws.update_project(project) # or project.save()

### List All Projects

To see what projects are currently stored in the workspace, we can list them like this.


In [ ]:
ws.list_projects()

### Search for a Project by Name

If we need to locate a specific project, we can search for it by name.


In [ ]:
ws.search_project("My Project")

### Delete a Project

If a project is no longer needed, we can delete it by its project ID.


In [ ]:
ws.delete_project(project.id)

### Create a Project Using ProjectModel

Alternatively, projects can be created by defining a `ProjectModel` with specific properties and adding it to the workspace. This approach can be useful for more programmatic setups.


In [ ]:
from evidently.ui.workspace import ProjectModel

project = ws.add_project(ProjectModel(name="My Project", description="Evidently Service example project"))
project

## Working with Reports and Runs

Next, let’s create a simple Evidently report. We’ll define a metric to check the minimum value of a specific column, and include a test to assert that this value should be greater than 5.


In [ ]:
from evidently.tests import gt
from evidently import Report
from evidently.metrics import MinValue

report = Report(metrics=[MinValue(column="col", tests=[gt(5)])])
report

Let’s generate some example data and run the report on it.

In [ ]:
import pandas as pd

data = pd.DataFrame({"col": [6, 10, 15]})
run = report.run(data)
run

### Add the Run to the Project

Now we can save this run into our project within the workspace.

In [ ]:
ws.add_run(project.id, run)

To view all runs associated with this project, we can list them.


In [ ]:
runs = ws.list_runs(project.id)
runs

If we need to retrieve a saved run and inspect its contents, we can load it like this.


In [ ]:
from evidently.core.report import Snapshot

Snapshot.load_model(ws.get_run(project.id, runs[0]))

Similarly, we can delete a specific run by its ID if it's no longer needed.


In [ ]:
ws.delete_run(project.id, runs[0])

To simulate multiple monitoring runs over time, we’ll generate several runs with slightly different random values and assign each a different timestamp.


In [ ]:
import random
import datetime

for i in range(1, 5):
    data = pd.DataFrame({"col": [i + random.randint(-2, 2) for _ in range(3)]})
    run = report.run(data, timestamp=datetime.datetime.now() + datetime.timedelta(days=-2 + i))
    ws.add_run(project.id, run)

Let’s list them to confirm they’re stored correctly.

In [ ]:
ws.list_runs(project.id)

### Start the Evidently UI Service

To explore our projects and reports in a web interface, you normally need to start the Evidently UI service from your command line like this:

```bash
evidently ui
```

This will run the service at [http://127.0.0.1:8000](http://127.0.0.1:8000).

For convenience, in the next cell we'll demonstrate how to launch the service directly from the notebook using a Python subprocess.
**Note:** This approach is handy for quick experiments, but it's generally recommended to run the service from your terminal in production or more persistent setups.


In [ ]:
import subprocess

# Start Evidently UI as a background process
process = subprocess.Popen(["evidently", "ui"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Store the process if you want to stop it later:
# process.terminate()

Now you can open [http://127.0.0.1:8000](http://127.0.0.1:8000) in your browser to explore your workspace and project.


### Working with RemoteWorkspace

The workflow is the same for a remote Evidently UI instance.
Here’s how you’d connect to a service running remotely (or locally in this case) using `RemoteWorkspace`.


In [ ]:
from evidently.ui.workspace import RemoteWorkspace

remote_ws = RemoteWorkspace("http://127.0.0.1:8000")
remote_ws

Once connected, we can search for and retrieve a project from the remote UI service.


In [ ]:
remote_project = remote_ws.search_project("My Project")[0]
remote_project

### Setting up dashboard

Finally, we can set up a dashboard panel to visualize metrics from our runs.
In this example, we’ll create a line plot panel showing the minimum value of the `"col"` column across different runs over time.


In [ ]:
from evidently.sdk.models import PanelMetric
from evidently.sdk.panels import line_plot_panel

remote_project.dashboard.add_panel(
    line_plot_panel(
                title="Minimum value",
                values=[
                    PanelMetric(
                        legend="minimum value",
                        metric="MinValue",
                        metric_labels={"column": "col"},
                    ),
                ],
                size="full",
            )
)

Now you can view your dashboard and metrics visualizations through the Evidently UI.
For more information, check the Evidently documentation at [Evidently AI Docs](https://docs.evidentlyai.com).
